In [18]:
import matplotlib.pyplot as plt
import networkx as nx
import heapq
import time
import pickle
import math

def manhattan_distance(node, goal):
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

def euclidean_distance(node, goal):
    return math.sqrt((node[0] - goal[0])**2 + (node[1] - goal[1])**2)

def max_distance(node, goal):
    return max(abs(node[0] - goal[0]), abs(node[1] - goal[1]))

def calculate_path_length(path):
    """
    Oblicza długość podanej ścieżki.

    :param path: Lista węzłów reprezentująca ścieżkę.
    :return: Długość ścieżki.
    """
    if not path:
        return 0

    length = 0
    for i in range(1, len(path)):
        # Obliczanie różnicy w położeniu x i y między kolejnymi węzłami
        dx = abs(path[i][0] - path[i-1][0])
        dy = abs(path[i][1] - path[i-1][1])
        
        # Zakładamy, że odległość między sąsiadującymi węzłami jest 1
        length += dx + dy

    return length

def a_star(graph, start, goal, heuristic):
    start_time = time.time()

    open_list = []
    heapq.heappush(open_list, (0, start))
    
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic(start, goal)
    
    while open_list:
        _, current = heapq.heappop(open_list)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            end_time = time.time()
            print(f'Czas działania algorytmu A*: {end_time - start_time}s')
            return path
        
        for neighbor in graph[current]:
            tentative_g_score = g_score[current] + graph[current][neighbor]
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                if neighbor not in [i[1] for i in open_list]:
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))



def dijkstra(graph, start, goal):
    start_time = time.time()
    queue = []
    heapq.heappush(queue, (0, start))
    
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    came_from = {}
    
    while queue:
        current_distance, current_node = heapq.heappop(queue)
        
        if current_node == goal:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            path.reverse()
            end_time = time.time()
            print(f'Czas działania algorytmu Dijkstry: {end_time - start_time}s')
            return path
        
        if current_distance > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                came_from[neighbor] = current_node
                heapq.heappush(queue, (distance, neighbor))

    return None

def draw_graph(graph, path=None):
    """
    Rysuje graf reprezentujący mapę pomieszczenia z uwzględnieniem optymalizacji wydajności.
    
    :param graph: Słownik reprezentujący graf (węzły jako klucze, sąsiedzi i odległości jako wartości).
    :param path: Lista węzłów reprezentująca ścieżkę do narysowania (opcjonalnie).
    """
    G = nx.Graph()
    
    for node, neighbors in graph.items():
        for neighbor, cost in neighbors.items():
            G.add_edge(node, neighbor, weight=cost)
    
    pos = {node: node for node in graph} 
    
    nx.draw_networkx_nodes(G, pos, node_size=2, node_color='lightblue')
    
    nx.draw_networkx_edges(G, pos, width=1.0)
    
    if len(graph) <= 100:
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    
    if path:
        path_edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2)
    
    plt.axis('off') 
    plt.show()

In [19]:
def add_obstacle(graph, start, width, height):
    """
    Dodaje przeszkodę do grafu, modyfikując wagi krawędzi.
    
    :param graph: Graf w postaci słownika.
    :param start: Pozycja początkowa przeszkody (x, y).
    :param width: Szerokość przeszkody.
    :param height: Wysokość przeszkody.
    """
    x_start, y_start = start
    for y in range(y_start, y_start + height):
        for x in range(x_start, x_start + width):
            if (x, y) in graph:
                for neighbor in list(graph[(x, y)].keys()):
                    graph[(x, y)][neighbor] = float('inf')
                    graph[neighbor][(x, y)] = float('inf')

In [20]:
def create_grid_graph(width, height):
    """
    Tworzy prostokątny graf o podanych wymiarach.
    
    :param width: Szerokość prostokąta (liczba kolumn).
    :param height: Wysokość prostokąta (liczba wierszy).
    :return: Graf w postaci słownika.
    """
    graph = {}
    
    for y in range(height):
        for x in range(width):
            graph[(x, y)] = {}
            
            if x > 0:
                graph[(x, y)][(x - 1, y)] = 1  # Lewo
            if x < width - 1:
                graph[(x, y)][(x + 1, y)] = 1  # Prawo
            if y > 0:
                graph[(x, y)][(x, y - 1)] = 1  # Góra
            if y < height - 1:
                graph[(x, y)][(x, y + 1)] = 1  # Dół

    return graph

In [21]:
import random
import numpy as np

def add_random_obstacles(graph, num_obstacles, max_width, max_height):
    """
    Dodaje losowe przeszkody do grafu.
    
    :param graph: Graf w postaci słownika.
    :param num_obstacles: Liczba przeszkód do dodania.
    :param max_width: Maksymalna szerokość przeszkody.
    :param max_height: Maksymalna wysokość przeszkody.
    """
    width, height = max(graph.keys())
    for _ in range(num_obstacles):
        obstacle_width = random.randint(1, max_width)
        obstacle_height = random.randint(1, max_height)
        x_start = random.randint(0, width - obstacle_width)
        y_start = random.randint(0, height - obstacle_height)
        add_obstacle(graph, (x_start, y_start), obstacle_width, obstacle_height)

def draw_map(graph, path=None, fname=None):
    """
    Rysuje mapę na podstawie grafu, zaznacza przeszkody czarnym kolorem i trasę czerwonym.
    
    :param graph: Graf w postaci słownika.
    :param path: Lista węzłów reprezentująca ścieżkę (opcjonalnie).
    :param fname: Nazwa pliku do zapisu mapy (opcjonalnie).
    """
    # Ustalanie rozmiarów mapy
    width = max(x for x, y in graph.keys()) + 1
    height = max(y for x, y in graph.keys()) + 1

    # Inicjalizacja mapy
    grid = np.ones((height, width, 3))  # Tworzenie białej mapy (1, 1, 1)

    # Rysowanie przeszkód
    for (x, y), neighbors in graph.items():
        if all(weight == float('inf') for weight in neighbors.values()):
            grid[y, x] = [0, 0, 0]  # Czarny kolor dla przeszkód

    grid[0, :, :] = [0, 0, 0]  # Górna krawędź
    grid[-1, :, :] = [0, 0, 0]  # Dolna krawędź
    grid[:, 0, :] = [0, 0, 0]  # Lewa krawędź
    grid[:, -1, :] = [0, 0, 0]  # Prawa krawędź

    # Rysowanie ścieżki, jeśli jest dostarczona
    if path:
        for (x, y) in path:
            grid[y, x] = [1, 0, 0]  # Czerwony kolor dla ścieżki

        # Rysowanie startu i celu
        

        start = path[0]
        goal = path[-1]
        grid[start[1], start[0]] = [0, 1, 0]  # Zielony kolor dla startu
        grid[goal[1], goal[0]] = [0, 0, 1]  # Niebieski kolor dla celu

    # Wyświetlanie lub zapis mapy
    if fname:
        
        plt.imsave(fname, grid)
    else:
        plt.imshow(grid)
        plt.grid(False)
        plt.show()




In [22]:
for i in range(5):
    graph = create_grid_graph(200, 50)
    add_random_obstacles(graph, 130, 10, 10)

    draw_map(graph, None, f'map_{i}.png')


    start = (0, 30)
    goal = (199, 20)

    heuristics={
        "manhattan" : manhattan_distance,
        "euclidean" : euclidean_distance,
        "max" : max_distance
    }

    print(f"iteracja: {i}")
    path2 = dijkstra(graph, start, goal)
    print(f"Długość trasy Dijkstra: {calculate_path_length(path2)}")
    draw_map(graph, path2, f'map_dijkstra_{i}.png')


    for k, v in heuristics.items():
        print(f"Funkcja obliczająca odległość w algorytmie A*: {k}")
        path1 = a_star(graph, start, goal, v)
        print(f"Długość trasy A*: {calculate_path_length(path1)}, { k }")
        draw_map(graph, path1, f'map_astar_{k}_{i}.png')


iteracja: 0
Czas działania algorytmu Dijkstry: 0.023378849029541016s
Długość trasy Dijkstra: 247
Funkcja obliczająca odległość w algorytmie A*: manhattan
Czas działania algorytmu A*: 0.1139984130859375s
Długość trasy A*: 247, manhattan
Funkcja obliczająca odległość w algorytmie A*: euclidean
Czas działania algorytmu A*: 0.10295534133911133s
Długość trasy A*: 247, euclidean
Funkcja obliczająca odległość w algorytmie A*: max
Czas działania algorytmu A*: 0.09905147552490234s
Długość trasy A*: 247, max
iteracja: 1
Czas działania algorytmu Dijkstry: 0.016851425170898438s
Długość trasy Dijkstra: 243
Funkcja obliczająca odległość w algorytmie A*: manhattan
Czas działania algorytmu A*: 0.1030118465423584s
Długość trasy A*: 243, manhattan
Funkcja obliczająca odległość w algorytmie A*: euclidean
Czas działania algorytmu A*: 0.12160587310791016s
Długość trasy A*: 243, euclidean
Funkcja obliczająca odległość w algorytmie A*: max
Czas działania algorytmu A*: 0.11968302726745605s
Długość trasy A*: 2